## Overview

The process of grounding involves integrating a language model (such as GPT) with specific data sources to enhance its performance and relevance. In the case of Bing Chat, Microsoft has employed this technique to create a more accurate and context-aware search experience by combining OpenAI’s GPT model with Bing search results.

Here’s how it works:

GPT Model: Large language models like GPT-4 are trained on data up to a certain point in time. For instance, ChatGPT’s training data stopped at some point in 2021.
Bing Data: Bing, on the other hand, has a vast index of web pages, ranking algorithms, and answer results. It provides relevant information based on user queries.
You can use variouse techniques to bridges the gap between GPT and Bing to combines the power of both to generate accurate and rich answers for your queries.

### Bing Search API:
The Bing Search API is a powerful tool provided by Microsoft, enabling developers to integrate Bing's search capabilities directly into their applications. This API allows access to a wide range of search results, including web pages, images, videos, and news, providing developers with a seamless way to incorporate Bing's search functionality into their own projects.

### Import python libararies

In [ ]:
from langchain.chains import TransformChain
import requests
import json
from openai import AzureOpenAI

### Get Environment variables
The provided code is importing the `load_dotenv` function from the `dotenv` module and using it to load environment variables from a `.env`

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")


### set up the Bing Search API
This Python code snippet is used to set up the Bing Search API.
- The `bing_search_url` variable is set to base URL for the Bing Search API.
- The `bing_subscription_key` variable is set to the value of the "BING_SUBSCRIPTION_KEY" environment variable. 

In [ ]:
bing_search_url = "https://api.bing.microsoft.com/v7.0/search"
bing_subscription_key = os.getenv("BING_SUBSCRIPTION_KEY")

### Get Guest name from the file saved in previous lab
To retrieve the guest name from the file saved in the previous lab, you can use the following code:

In [ ]:
# Open the file
with open('guest_name.txt', 'r') as f:
    guest = f.read()

# Print the content
print(guest)

## 1. Test Bing Search 
Lets'see what Bing Search returns as search results

### Define function to perform Bing search
This Python function, `search_bing`, is used to perform a search using the Bing Search API.

- The function takes one argument, `query`, which is the search query.
- The `headers` dictionary is created with the key "Ocp-Apim-Subscription-Key" and the value of `bing_subscription_key`. This is used to authenticate the request to the Bing Search API.
- The `requests.get` function sends a GET request to the Bing Search API.
- The `response.raise_for_status` method is called to raise an exception if the HTTP request returned an unsuccessful status code.
- A for loop is used to iterate over the search results. For each result, a dictionary is created with the keys "title", "link", and "snippet", and the corresponding values from the result. This dictionary is then appended to the `output` list.

In [ ]:
def search_bing(query):
    """
    Perform a bing search against the given query

    @param query: Search query
    @return: List of search results

    """
    headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}
    params = {"q": query, "textDecorations": False}
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results["webPages"]["value"]:
        output.append({"title": result["name"], "link": result["url"], "snippet": result["snippet"]})

    return json.dumps(output)

### Test Bing search and evaluate the search results
This Python code snippet is used to perform a search on Bing using the `search_bing` function with **"Who is [guest]?"**
Look at the results Bing Search is returning.

*You can experiment further and try your own queries*

In [ ]:
search_bing("Who is "+guest+"?")

## 2. Test GPT model without grounding
Let's see what GPT model returns without any grounding

### Connect to Azure OpenAI service
This Python code snippet is used to create a client for the Azure OpenAI service. The client is used to interact with the OpenAI API

In [ ]:
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2023-12-01-preview"
)
model =  os.getenv("AZURE_OPENAI_CHAT_MODEL")
print(model)

### Create a prompt for GPT model
*You can experiment further and try your own prompts*

In [ ]:
messages = [{"role": "user", "content": "Who is "+guest+"?"}]

### Send request to GPT model
This sends a request to the OpenAI API to generate a completion using the defined model and messages prompt created above.

In [ ]:
gpt_response = client.chat.completions.create(
            model=model,
            messages=messages,
        )  
    
bio = gpt_response.choices[0].message.content
print(bio)

## 3. Grounding with Function Calling
In Azure OpenAI, you can leverage the tools parameter to incorporate specific tools into your language model.
- Specify the relevant tools based on your use case. These tools can be predefined internal functions or custom tools you create.
- When making a request to the API, include the tools along with the user’s input (prompt).
- The model will consider these tools during response generation.

### Define Tools for GPT model, that include Bing search function
This Python code snippet defines `tools` that will be used by GPT model. 
here we are describing  a function that can be used for searching Bing to get up-to-date information.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_bing",
            "description": "Searches bing to get up-to-date information about people",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query",
                    }
                },
                "required": ["query"],
            },
        },
    }
]

### Send request to GPT model and see if there any tools calling
This sends a request to the OpenAI API to generate a completion using prompt and tools defined above.
You can see the result GPT model is returning with function calling

In [ ]:
response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
        )

response_message = response.choices[0].message
tool_calls = response_message.tool_calls
print(response_message)
print(tool_calls)

### Go through function calling and extract messages to see Bing search results
This Python code snippet is used to look the tool calls received and append the function's response to the conversation.

A for loop is used to iterate over the `tool_calls`. For each `tool_call`:
   - The corresponding function object is retrieved from `available_functions`
   - The function's `response_message` is appended to the `messages` list to extend the conversation.

In [ ]:
if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "search_bing": search_bing,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                query=function_args.get("query")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
print(messages)

### Ask GPT model to generate short bio of the person based on search results
We have extended our messages with Bing Search responses. 
Let's ask GPT model to answer **"Who is [guest]?"** question using additional information.

Compare with non-grounded answer from GPT model from the above

In [ ]:
second_response = client.chat.completions.create(
            model=model,
            messages=messages,
        )  # get a new response from the model where it can see the function response
    
bio = second_response.choices[0].message.content
print(bio)

### Save guest biography to file
This Python code snippet is used to write the guest's biography into `"bio.txt"` text file.

In [ ]:
# Specify the file path
file_path = "bio.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(bio)

## 4. Grounding with LangChain
Another approach to grounding can be LangChain Transform Chain. In this case, you clearly define that Bung Search query is part of your solution.
### Transform Chain in LangChain:
**Langchain** is a Python library that allows you to create powerful applications using language models and prompts.
Langchain's **Transform Chain** is a dynamic and versatile feature that facilitates the seamless transformation of text data. Through a series of customizable linguistic operations, developers can construct intricate text processing pipelines. This transformative chain empowers users to preprocess, analyze, and manipulate textual information efficiently, offering a flexible and scalable solution for diverse language processing tasks within the Langchain framework.

### Define function to perform Bing search
This Python function, `bing_grounding`, is used to perform a search using the Bing Search API.

In [ ]:
def bing_grounding(input_dict:dict) -> dict:
    print("Calling Bing Search API to get bio for guest...\n")
    search_term = input_dict["guest"]
    print("Search term is " + search_term)

    headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}
    params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    #print(search_results)

    # Parse out a bio.  
    bio = search_results["webPages"]["value"][0]["snippet"]
    
    print("Bio:\n")
    print(bio)
    print("\n")

    return {"bio": bio}


In [ ]:
bing_chain = TransformChain(input_variables=["guest"], output_variables=["bio"], transform=bing_grounding)
bio = bing_chain.run(guest)


### Save another version of guest biography if you prefere it
This Python code snippet is used to write the guest's biography into `"bio1.txt"` text file.


In [ ]:
# Specify the file path
file_path = "bio1.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(bio)